In [17]:
%load_ext autoreload
%autoreload 1  
# Automatically reload bioscout package
%aimport bioscout_tech_challenge

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from bioscout_tech_challenge import *
# Now any changes to your package will be automatically reloaded
import pandas as pd
import numpy as np

## Pre-Process Weather Data

Things to look at 
- extra_information column
- storage 
- autodetect header from mutliple files
- extract out of sensors to other tables
- add new data to existing tables
- write to sql db


Hardcode some data for testing.

In [19]:
weather_folder = r"data/tables/weather_data/"
devices_fn = "weather_devices.csv"
weather_fn = "weather_data_1.csv"


devices_df = read_csv_file(weather_folder+devices_fn)
weather_df = read_csv_file(weather_folder+weather_fn)

print(devices_df.columns)
print(weather_df.columns)

{}
{}
Index(['device_id', 'device_name', 'site_id', 'utc_offset_in_hours',
       'longitude', 'latitude'],
      dtype='object')
Index(['index', 'weather_reading_id', 'date_measured', 'device_id', 'voc',
       'pressure', 'extra_information'],
      dtype='object')


In [20]:
print(identify_header(weather_folder+weather_fn))

infer


In [21]:
merged_df = merge_weather_data(weather_df,devices_df)
#Check shape makes sense
print(devices_df.shape)
print(weather_df.shape)
print(merged_df.shape)


(16, 6)
(49999, 7)
(49999, 12)


In [22]:
# Check if any rows are missing corresponding device information

print(get_na_rows(merged_df,"index"))

null_rows = merged_df[merged_df['device_name'].isna()]
print(null_rows)



Empty DataFrame
Columns: [index, device_id]
Index: []
Empty DataFrame
Columns: [index, weather_reading_id, date_measured, device_id, voc, pressure, extra_information, device_name, site_id, utc_offset_in_hours, longitude, latitude]
Index: []


Lets try to combine multiple files automatically searching on a pattern. 

In [23]:
# Find all csv files in the weather folder

weather_csvs = find_csv_files(weather_folder,prefix="weather_data")
print(weather_csvs)

# Identify the header for each file
for csv in weather_csvs:
    print(identify_header(csv))



[PosixPath('/home/zach/repo/data.data_engineer_technical_challenge/data/tables/weather_data/weather_data_1.csv'), PosixPath('/home/zach/repo/data.data_engineer_technical_challenge/data/tables/weather_data/weather_data_2.csv'), PosixPath('/home/zach/repo/data.data_engineer_technical_challenge/data/tables/weather_data/weather_data_3.csv')]
infer
None
None


In [24]:
combined_df = combine_csv_files(weather_csvs,detect_header=True)
print(combined_df.head())

infer
{}


{'names': ['index', 'weather_reading_id', 'date_measured', 'device_id', 'voc', 'pressure', 'extra_information']}
{'names': ['index', 'weather_reading_id', 'date_measured', 'device_id', 'voc', 'pressure', 'extra_information']}
     index  weather_reading_id              date_measured  device_id      voc  \
0  7243411             9983574  2024-10-19 11:56:31+00:00        259   9400.0   
1  7243413             9983583  2024-10-19 11:56:51+00:00        265  10168.0   
2  7243636             9983588  2024-10-19 11:56:59+00:00        278   4287.0   
3  7243638             9983591  2024-10-19 11:57:02+00:00        262  10323.0   
4  7244430             9983595  2024-10-19 11:57:28+00:00        268  12693.0   

    pressure                                  extra_information  \
0  10246.000  {'VOCs': [{'Value': 9.4, 'Sensor': 'BME680'}],...   
1  10238.600  {'VOCs': [{'Value': 10.168, 'Sensor': 'BME680'...   
2   9869.601  {'VOCs': [{'Value': 4.287, 'Sensor': 'BME680'}...   
3   9862.400  {'VOC